In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os

In [3]:
print(tf.__version__)

2.3.0


In [4]:
emotion_dict = {0: "   Angry   ", 1: "Disgusted", 2: "  Fearful  ", 3: "   Happy   ", 4: "  Neutral  ", 5: "    Sad    ", 6: "Surprised"}
model = load_model("models/emotion.h5")

In [5]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

    grayed = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    n_faces = classifier.detectMultiScale(grayed,scaleFactor = 1.3, minNeighbors = 5)

    for x,y,w,h in n_faces:

        cv2.rectangle(frame, (x,y-50),(x+w,y+h+10), (0,255,0),2)

        cropped_img = grayed[y:y+h,x:x+w]

        input_img = np.expand_dims(np.expand_dims(cv2.resize(cropped_img, (48,48)), -1),0)

        prediction = model.predict(input_img)

        maxindex = int(np.argmax(prediction))

        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    cv2.imshow("Video Feed", cv2.resize(frame, (1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break